### **Import necessary python modules**

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Global constants


In [ ]:
""" This module defines physical and mathematical constants for atmospheric modeling. Careful for the units! """

MAXNZ = 50 # maximum number of layers
R_Gas = 8.3145e7 # Universal gas constant erg/(K mol)
StefBoltz = 5.6704e-5 # Stefan-Boltzmann constant erg/(cm^2 s K^4)
Avogadro = 6.02214179e23 # Avogadro's number molecules/mol
k_Boltz = 1.38064852e-16 # Boltzmann constant erg/K
Pi = np.pi

# Implementation of read_voyager.f
This code snippet talks about the way the input file from voyager_inputs in the main eddysed code has been handled.


In [ ]:
"""The input file from voyager_inputs in the main eddysed code.
 reads the following inputs:
 Pressure 
 Temperature
 Gravity
 Effective Temperature
 Number of layers edges

 It calculates the number of layers , effective convective heat flux (chf), 
 molecular weight of atmosphere (assumed constant at 2.2 g/mol), and builds the T and P layers and sub layers along with the creation of altitude layers (Z) and its sub layers.


 and returns the following outputs:

gravity, T_eff, num_layer_edges, p_layer_edges, t_layer_edges, z_layer_edges, num_layers,  p_layers, t_layers, z_layers, chf



"""


def read_voyager(input_file):

    input_data = []
    with open(input_file, "r") as f:
        lines = f.readlines()
        T_eff = float(lines[0])
        gravity = float(lines[1])
        num_layer_edges = int(lines[2])
        num_layers = num_layer_edges - 1
        for line in lines[4:]:
            parts = line.strip().split()
            input_data.append([float(x.replace("E", "e")) for x in parts])
    input_array = np.array(input_data)
    p_layer_edges = input_array[:, 0]
    t_layer_edges = input_array[:, 1]

    r_atmos = 8.3145e7/2.2

    z_layer_edges = np.zeros(num_layer_edges)
    z_layers = np.zeros(num_layers)
    p_layers = np.zeros(num_layers)
    t_layers = np.zeros(num_layers)

    for i in range(num_layers-1, -1, -1):
        itop = i
        ibot = i + 1
        dlnp = np.log(p_layer_edges[ibot]/p_layer_edges[itop])
        p_layers[i] = (p_layer_edges[itop] + p_layer_edges[ibot]) / 2
        scale_height = r_atmos * t_layer_edges[ibot] / gravity
        dz_layer = scale_height * dlnp
        z_layer_edges[i] = z_layers[ibot-1] + dz_layer
        dtdz = (t_layer_edges[itop] - t_layer_edges[ibot]) / dz_layer
        dz_pmid = scale_height * np.log(p_layer_edges[ibot]/p_layers[i])
        z_layers[i] = z_layer_edges[ibot] + dz_pmid
        t_layers[i] = t_layer_edges[ibot] + dtdz * dz_pmid

    chf = StefBoltz * T_eff**4 * np.ones(num_layers)

    return gravity, T_eff, num_layer_edges, p_layer_edges, t_layer_edges, z_layer_edges, num_layers,  p_layers, t_layers, z_layers, chf

# Implementation of layer.f

In [10]:
def layer(nsub_max, gas_name, kz_min, cloudf_min, mw_cloud, mw_atmos, f_sed, rho_p, supsat, sig_layer, t_layer, p_layer, t_top, t_bot, p_top, p_bot, chf_layer, gravity):

    # Set error return codes to zero
    status_r = 0
    status_q = 0

    # Number of levels of grid refinement used 
    nsub = 1

    # diameter of atmospheric molecule (cm) (Rosner, 2000)
    # (3.711e-8 for air, 3.798e-8 for N2, 2.827e-8 for H2)
    d_molecule = 2.827e-8

    # parameter in Lennard-Jones potential for viscosity (K) (Rosner, 2000)
    # (78.6 for air, 71.4 for N2, 59.7 for H2)
    eps_k = 59.7

    # specific gas constant for atmosphere (erg/(g K))
    r_atmos = R_Gas / mw_atmos

    # specific heat of atmosphere (erg/K/g)
    c_p = 7./2. * r_atmos

    # pressure thickness of layer
    dp_layer = p_bot - p_top
    dlnp = np.log( p_bot/p_top )

    # temperature gradient 
    dtdlnp = ( t_top - t_bot ) / dlnp
    lapse_ratio = ( t_bot - t_top ) / dlnp / ( 2./7.*t_layer )

    # atmospheric density (g/cm^3)
    rho_atmos = p_layer / ( r_atmos * t_layer )

    # atmospheric scale height (cm)
    scale_h = r_atmos * t_layer / gravity

    # convective mixing length scale (cm): no less than 1/10 scale height
    mixl = max( 0.1, lapse_ratio ) * scale_h

    # mixing length = scale height matches Lunine (1989) model
    mixl = scale_h

    # scale factor for eddy diffusion: 1/3 is baseline
    scalef_kz = 1./3.

    # vertical eddy diffusion coefficient (cm^2/s)
    # from Gierasch and Conrath (1985)
    kz = scalef_kz * scale_h * (mixl/scale_h)**(4/3) *( ( r_atmos*chf_layer) / ( rho_atmos*c_p ) )**(1/3)

    # no less than minimum value (for radiative regions)
    kz = max( kz, kz_min )

    # convective velocity scale (cm/s)
    w_convect = kz / mixl

    # cloud fractional coverage
    cloudf = cloudf_min + max( 0, min( 1, 1-lapse_ratio )) * ( 1 - cloudf_min )

    # atmospheric number density (molecules/cm^3)
    n_atmos = p_layer / ( k_Boltz*t_layer )

    # atmospheric mean free path (cm)
    mfp = 1. / ( np.sqrt(2.)*n_atmos*Pi*d_molecule**2 )

    # atmospheric viscosity (dyne s/cm^2)
    visc = 5./16.*np.sqrt( Pi*k_Boltz*t_layer*(mw_atmos/Avogadro)) / ( Pi*d_molecule**2 ) / ( 1.22 * ( t_layer / eps_k )**(-0.16) )

    # Start from the top of convergence layer in the layer.f file

    return None

# Implementation of eddysed.f

In [7]:
def eddysed(gravity, T_eff, kz_min, cloudf_min, nsub_max, supsat, mw_atmos, do_virtual, num_layers, z_layers, z_layer_edges, p_layers, p_layer_edges, t_layers, t_layer_edges, chf, gas_name, gas_mmr, gas_mw, rho_p, sig, f_sed):
    
    # Initialization:
    cloud_base = 0
    report_status_r = True
    report_status_q = True

    # Calculate indices of bottom and top of domain
    if z[1] > z[0]:
        k_bottom = 0
        k_top = num_layers - 1
        increment = 1
    else:
        k_bottom = num_layers - 1
        k_top = 0
        increment = -1

    pass